In [1]:
# load the dataset from the csv file using pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
from keras import optimizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

# load the dataset from the csv file using pandas
df = pd.read_csv('diabetes.csv')
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# Replace the 0 values with NA except for the [Pregnancies, Outcome, DiabetesPedigreeFunction, Age] columns
df_copy = df.copy(deep = True)
df_copy[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = df_copy[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.NaN)
df_copy.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [4]:
# impute missing values with KNN imputer
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_filled = imputer.fit_transform(df_copy)
df_filled = pd.DataFrame(df_filled, columns = df_copy.columns)
df_filled.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,169.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,58.6,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,25.8,164.6,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0


In [5]:
# check if there are any null values
df_filled.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
# scale the data using min_max except for outcome column
df_copy_scaled = df_copy.copy(deep = True)
df_copy_scaled[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']] = df_copy_scaled[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [7]:
# Split the data into training and test sets
X = df_copy_scaled.drop(columns = ['Outcome'])
y = df_copy_scaled['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=5)

In [17]:
# create a deep learning model with 3 hidden layers
model = Sequential([
    Dense(8, activation = 'relu', input_shape = (8,)),
    Dense(4, activation = 'relu'),
    Dense(2, activation = 'relu'),
    Dense(1, activation = 'sigmoid'),
])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
def create_model():
    model = Sequential([
        Dense(8, activation = 'relu', input_shape = (8,)),
        Dense(4, activation = 'relu'),
        Dense(2, activation = 'relu'),
        Dense(1, activation = 'sigmoid'),
    ])
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [14]:
y_train.dtype

dtype('int64')

In [9]:
# train the model
earlystoping = EarlyStopping(monitor = 'loss', mode = 'min', verbose = 1, patience = 100)
model.fit(X_train, y_train, epochs = 1000, validation_split = 0.1, verbose = 2, callbacks=[earlystoping])

Epoch 1/1000


2023-09-26 05:13:25.403666: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


20/20 - 1s - loss: 0.6883 - accuracy: 0.6425 - val_loss: 0.6812 - val_accuracy: 0.6857 - 755ms/epoch - 38ms/step
Epoch 2/1000
20/20 - 0s - loss: 0.6808 - accuracy: 0.6425 - val_loss: 0.6708 - val_accuracy: 0.6857 - 166ms/epoch - 8ms/step
Epoch 3/1000
20/20 - 0s - loss: 0.6736 - accuracy: 0.6425 - val_loss: 0.6612 - val_accuracy: 0.6857 - 160ms/epoch - 8ms/step
Epoch 4/1000
20/20 - 0s - loss: 0.6671 - accuracy: 0.6425 - val_loss: 0.6536 - val_accuracy: 0.6857 - 163ms/epoch - 8ms/step
Epoch 5/1000
20/20 - 0s - loss: 0.6623 - accuracy: 0.6425 - val_loss: 0.6460 - val_accuracy: 0.6857 - 159ms/epoch - 8ms/step
Epoch 6/1000
20/20 - 0s - loss: 0.6586 - accuracy: 0.6425 - val_loss: 0.6405 - val_accuracy: 0.6857 - 155ms/epoch - 8ms/step
Epoch 7/1000
20/20 - 0s - loss: 0.6561 - accuracy: 0.6425 - val_loss: 0.6364 - val_accuracy: 0.6857 - 156ms/epoch - 8ms/step
Epoch 8/1000
20/20 - 0s - loss: 0.6543 - accuracy: 0.6425 - val_loss: 0.6337 - val_accuracy: 0.6857 - 155ms/epoch - 8ms/step
Epoch 9/1000

KeyboardInterrupt: 

In [15]:
model.evaluate(X_test, y_test, verbose = 2)

3/3 - 0s - loss: 0.6249 - accuracy: 0.6883 - 60ms/epoch - 20ms/step


[0.6249444484710693, 0.6883116960525513]

In [16]:
model.predict(X_test)

3/3 [==============================] - 0s 7ms/step


array([[0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.35666358],
       [0.356

In [18]:
# do k-fold cross validation
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 5)
cvscores = []
for train, test in kfold.split(X, y):
    model = create_model()
    earlystoping = EarlyStopping(monitor = 'loss', mode = 'min', verbose = 1, patience = 100)
    model.fit(X.iloc[train], y.iloc[train], epochs = 1000, batch_size = 32, verbose = 2, callbacks=[earlystoping])
    scores = model.evaluate(X.iloc[test], y.iloc[test], verbose = 0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

Epoch 1/1000
22/22 - 0s - loss: 0.6893 - accuracy: 0.6512 - 446ms/epoch - 20ms/step
Epoch 2/1000
22/22 - 0s - loss: 0.6810 - accuracy: 0.6512 - 124ms/epoch - 6ms/step
Epoch 3/1000
22/22 - 0s - loss: 0.6737 - accuracy: 0.6512 - 121ms/epoch - 5ms/step
Epoch 4/1000
22/22 - 0s - loss: 0.6670 - accuracy: 0.6512 - 119ms/epoch - 5ms/step
Epoch 5/1000
22/22 - 0s - loss: 0.6618 - accuracy: 0.6512 - 119ms/epoch - 5ms/step
Epoch 6/1000
22/22 - 0s - loss: 0.6569 - accuracy: 0.6512 - 118ms/epoch - 5ms/step
Epoch 7/1000
22/22 - 0s - loss: 0.6536 - accuracy: 0.6512 - 124ms/epoch - 6ms/step
Epoch 8/1000
22/22 - 0s - loss: 0.6511 - accuracy: 0.6512 - 121ms/epoch - 5ms/step
Epoch 9/1000
22/22 - 0s - loss: 0.6494 - accuracy: 0.6512 - 119ms/epoch - 5ms/step
Epoch 10/1000
22/22 - 0s - loss: 0.6486 - accuracy: 0.6512 - 119ms/epoch - 5ms/step
Epoch 11/1000
22/22 - 0s - loss: 0.6476 - accuracy: 0.6512 - 121ms/epoch - 6ms/step
Epoch 12/1000
22/22 - 0s - loss: 0.6473 - accuracy: 0.6512 - 121ms/epoch - 6ms/step


KeyboardInterrupt: 

In [ ]:
# using library and do k-fold cross validation
